In [130]:

# For data processing
import pandas as pd
import numpy as np
import math
#from statsmodels.stats.weightstats import ztest
pd.set_option("display.max_rows", 20)
pd.set_option("display.max_columns", 30)
pd.options.mode.chained_assignment = None

# For data vis
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style = 'darkgrid')

import warnings
warnings.filterwarnings('ignore')


In [43]:
%matplotlib inline

In [131]:
df1 = pd.read_csv("DataScientist.csv", index_col = 0  )

df2 = pd.read_csv("DataAnalyst.csv",index_col = 0    )

#df1
#df2

In [142]:
df = pd.merge(df1,df2, how = 'outer')
#deleting the undesirable column 1:
df.columns = df.columns.str.strip()


#df

In [143]:
df.shape


(6161, 16)

In [144]:
df.nunique()

index                3909
Job Title            3014
Salary Estimate       232
Job Description      5309
Rating                 38
Company Name         2926
Location              315
Headquarters          703
Size                    9
Founded               191
Type of ownership      15
Industry              105
Sector                 26
Revenue                14
Competitors           547
Easy Apply              2
dtype: int64

In [145]:
(df == '-1').sum()

index                   0
Job Title               0
Salary Estimate         1
Job Description         0
Rating                  0
Company Name            0
Location                0
Headquarters          412
Size                  392
Founded                 0
Type of ownership     392
Industry              899
Sector                899
Revenue               392
Competitors          4492
Easy Apply           5917
dtype: int64

## Limpieza básica
* Eliminar duplicados y completar valores NaN.
* Eliminar primera columna (índice falso) y filas sin salario.
* Restablecer índice

In [147]:
#Checking columns names:
#df.columns
#dropping duplicates rows
df=df.drop_duplicates()
#deleting row without salary: our entirely analysis depends on salary info.
df = df[df['Salary Estimate'] != '-1']
#resetting index
df=df.reset_index(drop=True)
#df.isnull().sum() #This line count how many nan data are in each column of the df
#filling nan data with '-1'
df = df.fillna('-1')
df

,index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply
0,0.0,Senior Data Scientist,$111K-$181K (Glassdoor est.),"ABOUT HOPPER\n\nAt Hopper, we’re on a mission ...",3.5,Hopper\n3.5,"New York, NY","Montreal, Canada",501 to 1000 employees,2007,Company - Private,Travel Agencies,Travel & Tourism,Unknown / Non-Applicable,-1,-1
1,1.0,"Data Scientist, Product Analytics",$111K-$181K (Glassdoor est.),"At Noom, we use scientifically proven methods ...",4.5,Noom US\n4.5,"New York, NY","New York, NY",1001 to 5000 employees,2008,Company - Private,"Health, Beauty, & Fitness",Consumer Services,Unknown / Non-Applicable,-1,-1
2,2.0,Data Science Manager,$111K-$181K (Glassdoor est.),Decode_M\n\nhttps://www.decode-m.com/\n\nData ...,-1.0,Decode_M,"New York, NY","New York, NY",1 to 50 employees,-1,Unknown,-1,-1,Unknown / Non-Applicable,-1,True
3,3.0,Data Analyst,$111K-$181K (Glassdoor est.),Sapphire Digital seeks a dynamic and driven mi...,3.4,Sapphire Digital\n3.4,"Lyndhurst, NJ","Lyndhurst, NJ",201 to 500 employees,2019,Company - Private,Internet,Information Technology,Unknown / Non-Applicable,"Zocdoc, Healthgrades",-1
4,4.0,"Director, Data Science",$111K-$181K (Glassdoor est.),"Director, Data Science - (200537)\nDescription...",3.4,United Entertainment Group\n3.4,"New York, NY","New York, NY",51 to 200 employees,2007,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"BBDO, Grey Group, Droga5",-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6155,-1,RQS - IHHA - 201900004460 -1q Data Security An...,$78K-$104K (Glassdoor est.),Maintains systems to protect data from unautho...,2.5,"Avacend, Inc.\n2.5","Denver, CO","Alpharetta, GA",51 to 200 employees,-1,Company - Private,Staffing & Outsourcing,Business Services,Unknown / Non-Applicable,-1,-1
6156,-1,Senior Data Analyst (Corporate Audit),$78K-$104K (Glassdoor est.),Position:\nSenior Data Analyst (Corporate Audi...,2.9,Arrow Electronics\n2.9,"Centennial, CO","Centennial, CO",10000+ employees,1935,Company - Public,Wholesale,Business Services,$10+ billion (USD),"Avnet, Ingram Micro, Tech Data",-1
6157,-1,"Technical Business Analyst (SQL, Data analytic...",$78K-$104K (Glassdoor est.),"Title: Technical Business Analyst (SQL, Data a...",-1.0,Spiceorb,"Denver, CO",-1,-1,-1,-1,-1,-1,-1,-1,-1
6158,-1,"Data Analyst 3, Customer Experience",$78K-$104K (Glassdoor est.),Summary\n\nResponsible for working cross-funct...,3.1,Contingent Network Services\n3.1,"Centennial, CO","West Chester, OH",201 to 500 employees,1984,Company - Private,Enterprise Software & Network Solutions,Information Technology,$25 to $50 million (USD),-1,-1


In [150]:
#Since cells were found to have a different salary format, they should be highlighted.
#So two new columns with 1 for the detected cells and 0 for the normal cells are needed (one for each case):
df["hourly"] = df['Salary Estimate'].apply(lambda x: 1 if 'per hour' in x.lower() else 0)
df["employer_provided"] = df['Salary Estimate'].apply(lambda x: 1 if 'employer provided salary' in x.lower() else 0)

#Creating a new dataframe with the salary data and processed it:
    #Here we are deleting those rows without the salary data.
df = df[df['Salary Estimate'] != '-1']
    #Here is taken everything before the '(' ignoring the rest and pasting that in the serie "salary".
salary = df["Salary Estimate"].apply(lambda x: x.split('(')[0])
    #Here are erased the 'K' and '$
minus_Kd = salary.apply(lambda x: x.replace('K','').replace('$',''))
    #Here are replaced the cases previously detected with a blank space instead.
min_hr = minus_Kd.apply(lambda x: x.lower().replace('per hour','').replace('employer provided salary:',''))

#Now that only numbers are left, separated with a '-', split them and put them in two different dataframe will help to calculate the average
    #This doesn't take into consideration the data presented in an hourly form:
df['min_salary'] = min_hr.apply(lambda x: int(x.split('-')[0]))
df['max_salary'] = min_hr.apply(lambda x: int(x.split('-')[1]))
    #calculating average:

#Here, Ken Jee forgot to take into consideration those salary values that where given in an hourly form. Maybe, I should had declared a serie and do it there. However, it works anyway:
    #It is logical to think tha the companies that are hiring telling an hourly salary could not be hiring annually, hiring by season, or are not sure about investment in this area, so  the salaries could be irregular (generally worst)
        #Later if I don't want to add them in the EDA step, then there we got the "hourly" column
            #Why I multiply by 1.92? 8 hours, 5 days, 4 weeks, 12 months: 8*5*4*12=1920... The values in the column are in miles, so I need to divide by 1000.
df['min_salary']= df.apply(lambda x: x['min_salary'] if x['hourly']==0 else (int)(x['min_salary']*1.92), axis = 1)
df['max_salary']= df.apply(lambda x: x['max_salary'] if x['hourly']==0 else (int)(x['max_salary']*1.92), axis = 1)
        
    #Finally, averaging column is calculated:
df['avg_salary'] = (df.min_salary+df.max_salary)/2
#df

In [151]:
#Saco cuales son los nombres de las columnas
#df_result.columns.values

In [153]:
#df3= df_result[['Job Title', 'Salary Estimate','Job Description', 'Company Name', 'Location','Headquarters', 'Size', 'Founded', 'Type of ownership', 'Industry','Sector', 'Revenue']]

#df3.info()
#df3.head()

In [154]:
df.isnull().sum()

index                0
Job Title            0
Salary Estimate      0
Job Description      0
Rating               0
                    ..
hourly               0
employer_provided    0
min_salary           0
max_salary           0
avg_salary           0
Length: 21, dtype: int64

In [166]:
#Taking into account that the Company_Name  column has not only the name of the company, but also the information of the ratings, it is necessary to remove ir the latter.
#If you know the glassdoor_scraping.py code you will know that when there is no rating, the cell will be filled with "-1", but when there is rating the content of the cell will be cero above (this also can be seen just looking the data).
#Finally, the rating (when exist) has 3 characters (number dot number)
#So, the next line is telling that when the Rating column has a value greater than zero, the company name will have 3 characters removed.
    #the "axis = 1" parameter says that the data will be searched in the same row
df['company_txt']= df.apply(lambda x: x['Company Name'] if x['Rating']<0 else x['Company Name'][:-3], axis = 1)

#If we watch what we have in the company names treated column:
df['company_txt']
#It is quite evident that each cell has "/n" characters at the end. Then we should remove it.

#Ken Jee used this:
df['company_txt'] = df.company_txt.apply(lambda x: x.replace('\n',''))
#I think this is better:
df['company_txt'] = df['company_txt'].str[:-1]
df['company_txt']

0                            Hoppe
1                           Noom U
2                          Decode_
3                  Sapphire Digita
4        United Entertainment Grou
                   ...            
6155                  Avacend, Inc
6156              Arrow Electronic
6157                       Spiceor
6158    Contingent Network Service
6159                     SCL Healt
Name: company_txt, Length: 6160, dtype: object

In [176]:
df = pd.read_csv('data_cleaned_2020.csv', sep = r',')
df.columns = df.columns.str.strip()
df.head(2)

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply,...,datarobot_yn,hana_yn,mongo_yn,trifacta_yn,minitab_yn,kafka_yn,microstrategy_yn,google_an_yn,spss_yn,job_simp,seniority_by_title,seniority_by_description,seniority,desc_len,desc_len_words
0,Senior Data Scientist,$111K-$181K (Glassdoor est.),"about hopper at hopper, we’re on a mission to...",3.5,Hopper\n3.5,"New York, NY","Montreal, Canada",501 to 1000 employees,2007,Company - Private,Travel Agencies,Travel & Tourism,Unknown / Non-Applicable,-1,-1,...,0,0,0,0,0,0,0,0,0,data scientist,sr,na,sr,3417,526
1,"Data Scientist, Product Analytics",$111K-$181K (Glassdoor est.),"at noom, we use scientifically proven methods ...",4.5,Noom US\n4.5,"New York, NY","New York, NY",1001 to 5000 employees,2008,Company - Private,"Health, Beauty, & Fitness",Consumer Services,Unknown / Non-Applicable,-1,-1,...,0,0,0,0,0,0,0,0,0,data scientist,na,ssr,ssr,2350,367


In [177]:
df.columns

Index(['Job Title', 'Salary Estimate', 'Job Description', 'Rating',
       'Company Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors',
       'Easy Apply', 'hourly', 'employer_provided', 'min_salary', 'max_salary',
       'avg_salary', 'company_txt', 'job_state', 'age', 'python_yn', 'R_yn',
       'spark_yn', 'aws_yn', 'excel_yn', 'sql_yn', 'sas_yn', 'd3js_yn',
       'julia_yn', 'jupyter_yn', 'keras_yn', 'matlab_yn', 'matplotlib_yn',
       'pytorch_yn', 'scikit_yn', 'tensor_yn', 'weka_yn', 'selenium_yn',
       'hadoop_yn', 'tableau_yn', 'bi_yn', 'bigml_yn', 'rapidminer_yn',
       'flink_yn', 'datarobot_yn', 'hana_yn', 'mongo_yn', 'trifacta_yn',
       'minitab_yn', 'kafka_yn', 'microstrategy_yn', 'google_an_yn', 'spss_yn',
       'job_simp', 'seniority_by_title', 'seniority_by_description',
       'seniority', 'desc_len', 'desc_len_words'],
      dtype='object')